In [5]:
!pip install -q kaggle
from google.colab import files
files.upload()


Saving kaggle.json to kaggle (1).json


{'kaggle (1).json': b'{"username":"arpitkumargupta","key":"bba778e802edc834f1daab046ada35e0"}'}

In [6]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [7]:
!kaggle datasets download -d kritanjalijain/amazon-reviews

Dataset URL: https://www.kaggle.com/datasets/kritanjalijain/amazon-reviews
License(s): CC0-1.0
amazon-reviews.zip: Skipping, found more recently modified local copy (use --force to force download)


In [8]:
!unzip amazon-reviews.zip
!ls

Archive:  amazon-reviews.zip
replace amazon_review_polarity_csv.tgz? [y]es, [n]o, [A]ll, [N]one, [r]ename:  amazon_review_polarity_csv.tgz   glove.6B.300d.txt   kaggle.json
 amazon-reviews.zip		  glove.6B.50d.txt    sample_data
 glove.6B.100d.txt		  glove.6B.zip	      test.csv
 glove.6B.200d.txt		 'kaggle (1).json'    train.csv


In [11]:
import pandas as pd
train_df = pd.read_csv('train.csv',header=None,nrows=100000)

train_df.rename(columns={0:'Label',1:'Title',2:'Review'},inplace=True)
train_df=train_df.dropna()
train_df=train_df.reset_index(drop=True)
print(train_df.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99995 entries, 0 to 99994
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Label   99995 non-null  int64 
 1   Title   99995 non-null  object
 2   Review  99995 non-null  object
dtypes: int64(1), object(2)
memory usage: 2.3+ MB
None


In [13]:
positive_df=train_df[train_df['Label']==2]
negative_df=train_df[train_df['Label']==1]
print(train_df.head(100))

# # print(train_df.iloc[:1000,:][train_df['Review'].iloc[:1000].str.contains(r'\d')].head().to_string())
print(train_df['Review'].str.len().max())


#Better after tokenisation:

# l_a=[25*x for x in range(13,24)]
# lengths=[0,100,110,120,130,140,150,160]
# lengths.extend([20*x for x in range(9,16)])
# l_b=[600,630,660,700,750,800,850,900,1025]
# lengths.extend(l_a)
# lengths.extend(l_b)


# for i in range(1,35):
#     p=positive_df[(positive_df['Review'].str.len()<lengths[i])&(positive_df['Review'].str.len()>=lengths[i-1])].index.size
#     q=negative_df[(negative_df['Review'].str.len()<lengths[i])&(negative_df['Review'].str.len()>=lengths[i-1])].index.size
#     print(f"Length of sequence: {lengths[i-1]} to {lengths[i]} Number of positive reviews: {p} Number of Negative reviews: {q}, positive_class_percentage:{p/(q+p)}")


!pip  install symspellpy
!pip install pandarallel

from pandarallel import pandarallel
pandarallel.initialize(progress_bar=True, nb_workers=4)

import inflect
import re
import unicodedata
from symspellpy.symspellpy import SymSpell, Verbosity

import nltk
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

func=inflect.engine()
lemmatiser=WordNetLemmatizer()
sym_spell = SymSpell(max_dictionary_edit_distance=2, prefix_length=7)
sym_spell.load_dictionary("frequency_dictionary_en_82_765.txt",0,1)

nltk.download('stopwords')
nltk.download('punkt_tab')
nltk.download('averaged_perceptron_tagger_eng')
nltk.download('wordnet')

stop_words = set(stopwords.words('english'))
keep_words = {
    'not', 'no', 'never', 'nor', 'very', 'too', 'so', 'just', 'really',
    'maybe', 'perhaps', 'definitely', 'surely', 'even', 'only', 'still',
    'yet', 'most', 'more', 'less', 'least', 'better', 'worse', 'best', 'worst',
    'always', 'already', 'clearly', 'almost', 'barely', 'hardly', 'scarcely',
    'totally', 'absolutely', 'completely', 'entirely', 'strongly', 'immensely',
        'certainly','undoubtedly', 'obviously','oh', 'wow', 'ugh','aha','oops','again','good', 'bad', 'much' }

contractions_dict = {
    "can't": "cannot", "won't": "will not", "n't": " not", "'re": " are", "'s": " is","'d": " would","'ll": " will","'t": " not","'ve": " have", "'m": " am"}

stop_words-=keep_words

DIGIT_RE = re.compile(r'\b\d{1,10}\b')
WHITESPACE_RE = re.compile(r'\s+')
NON_WORD_RE = re.compile(r'[^\w\s!?]')

def complete_preprocessor(text):

    def correct_spelling(text):
      suggestions = sym_spell.lookup_compound(text, max_edit_distance=2)
      if suggestions:
          return suggestions[0].term
      return text

    def num_to_text(text):
        def f(match):
            txt=match.group()
            return func.number_to_words(int(txt))
        return re.sub(DIGIT_RE,f,text)

    def fix_contractions(txt):
        for x,y in contractions_dict.items():
            pattern = re.compile(r'\b' + re.escape(x) + r'\b', flags=re.IGNORECASE)
            txt = pattern.sub(y, txt)
        return txt


    def get_wordnet_tag(tb_tag):
        if tb_tag.startswith('J'):
            return 'a'
        elif tb_tag.startswith('V'):
            return 'v'
        elif tb_tag.startswith('R'):
            return 'r'
        elif tb_tag.startswith('N'):
            return 'n'
        else:
            return None # To take care of relevant punctuations

    def lemmatise(ls):
        return [lemmatiser.lemmatize(token,get_wordnet_tag(tb_tag)) if get_wordnet_tag(tb_tag) else token for token, tb_tag in ls if token not in stop_words]

    if not isinstance(text,str):
        return text
    text=text.lower().strip()
    text=re.sub(WHITESPACE_RE, ' ', text)

    text=unicodedata.normalize('NFKD', text)
    text=num_to_text(text)
    text=re.sub(NON_WORD_RE,'', text)
    text=fix_contractions(text)
    text=correct_spelling(text)

    word_tokens=word_tokenize(text)
    tags=pos_tag(word_tokens)
    return lemmatise(tags)

#Rare Words later

train_df['Title']=train_df['Title'].parallel_apply(complete_preprocessor)
train_df['Review']=train_df['Review'].parallel_apply(complete_preprocessor)

train_df=train_df[(train_df['Title'].apply(len)>0)&(train_df['Review'].apply(len)>0)]
train_df=train_df.reset_index(drop=True)
train_df['Label']-=1

    Label                               Title  \
0       1              [stun, even, nongamer]   
1       1  [best, soundtrack, ever, anything]   
2       1                             [amaze]   
3       1             [excellent, soundtrack]   
4       1  [remember, pull, jaw, floor, hear]   
..    ...                                 ...   
95      0            [very, not, worth, time]   
96      1            [very, fun, educational]   
97      0                  [ludicrous, silly]   
98      1                          [artistry]   
99      0                           [caution]   

                                               Review  
0   [sound, track, beautiful, paint, senery, mind,...  
1   [im, read, lot, review, say, best, game, sound...  
2   [soundtrack, favorite, music, time, hand, inte...  
3   [truly, like, soundtrack, enjoy, video, game, ...  
4   [youve, play, game, know, divine, music, every...  
..                                                ...  
95  [book, wriiten,

2025-06-10 15:37:53,976: E symspellpy.symspellpy] Dictionary file not found at frequency_dictionary_en_82_765.txt.
ERROR:symspellpy.symspellpy:Dictionary file not found at frequency_dictionary_en_82_765.txt.


INFO: Pandarallel will run on 4 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [21]:
!ls
!rm glove.6B.zip.1 glove.6B.zip
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip glove.6B.zip
!ls

glove_input_file = 'glove.6B.100d.txt'

from nltk.corpus import sentiwordnet
nltk.download('sentiwordnet')



 amazon_review_polarity_csv.tgz   glove.6B.300d.txt   kaggle.json
 amazon-reviews.zip		  glove.6B.50d.txt    sample_data
 glove.6B.100d.txt		  glove.6B.zip	      test.csv
 glove.6B.200d.txt		 'kaggle (1).json'    train.csv
rm: cannot remove 'glove.6B.zip.1': No such file or directory
--2025-06-10 15:55:18--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2025-06-10 15:55:19--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2025-06-10 15:55:19--  https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downlo

[nltk_data] Downloading package sentiwordnet to /root/nltk_data...
[nltk_data]   Package sentiwordnet is already up-to-date!


True

In [35]:
# new_lengths=[0,100]
# new_lengths.extend([50*x for x in range(3,22)])

positive_df=train_df[train_df['Label']==2]
negative_df=train_df[train_df['Label']==1]

print(positive_df[positive_df['Review'].apply(len)==0].index.size)
print(negative_df[negative_df['Review'].apply(len)==0].index.size)
# for i in range(1,21):
#     p=positive_df[(positive_df['Review'].apply(len)<new_lengths[i])&(positive_df['Review'].apply(len)>=new_lengths[i-1])].index.size
#     q=negative_df[(negative_df['Review'].apply(len)<new_lengths[i])&(negative_df['Review'].apply(len)>=new_lengths[i-1])].index.size
#     print(f"Length of sequence: {new_lengths[i-1]} to {new_lengths[i]} Number of positive reviews: {p} Number of Negative reviews: {q}, positive_class_percentage:{p/(q+p)}")

from collections import Counter
import torch
import numpy as np
from torch.nn.utils.rnn import pad_sequence

def build_vocab(df):
    counter=Counter()
    cols=['Title','Review']
    for col in cols:
        df[col].apply(lambda word_tokens: counter.update(word_tokens))
    vocab={'UNK':1,'PAD':0,'POS':2,'NEG':3}
    index=4
    for word,freq in counter.items():
        if freq>=1:
            vocab[word]=index
            index+=1
    return vocab

def int_tokens(df,vocab):

    def get_int_tokens(ls):
        return [vocab.get(token) if token in vocab else vocab.get('UNK') for token in ls]

    cols=['Title','Review']
    for col in cols:
        df[col]=df[col].apply(lambda x : get_int_tokens(x))

def seq_pad(df,pad_value=0):
    df['Title']=df['Title'].apply(lambda x: torch.tensor(x,dtype=torch.long))
    df['Review']=df['Review'].apply(lambda x: torch.tensor(x,dtype=torch.long))

    a=(torch.tensor(df['Label'].values,dtype=torch.long))
    Title_tensor = pad_sequence(df['Title'].tolist(), batch_first=True, padding_value=pad_value)
    Review_tensor = pad_sequence(df['Review'].tolist(), batch_first=True, padding_value=pad_value)
    return a,Title_tensor,Review_tensor


# indices=train_df.index
# np.random.shuffle(indices)

train_df['Label']+=1
df=train_df.head(50000).copy()
print(df[df['Label']==1].index.size)
print(train_df['Label'].describe())
Vocab=build_vocab(df)
int_tokens(df,Vocab)
print(df.info())
print(df)
label_tensor,title_tensor,review_tensor=seq_pad(df)


0
0
25536
count    99712.000000
mean         0.513298
std          0.499826
min          0.000000
25%          0.000000
50%          1.000000
75%          1.000000
max          1.000000
Name: Label, dtype: float64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Label   50000 non-null  int64 
 1   Title   50000 non-null  object
 2   Review  50000 non-null  object
dtypes: int64(1), object(2)
memory usage: 1.1+ MB
None
       Label                    Title  \
0          1                [4, 5, 6]   
1          1            [7, 8, 9, 10]   
2          1                     [11]   
3          1                  [12, 8]   
4          1     [13, 14, 15, 16, 17]   
...      ...                      ...   
49995      1      [89, 913, 365, 688]   
49996      0      [1309, 17170, 3485]   
49997      1           [31, 213, 468]   
49998      0  [58, 559, 89, 285, 

1.Better to replace numbers with notation rather than to drop them;
Ensured class imbalance as well as bucket size consistency (ensures no bias, learned stratified bucketing) and so no need for Resampling or Algorithmic manipulations to consider it , can go ahead directly with training with bucketing, with both internal shuffling as well as shuffling bucket order as well;
Using titles: Do not just concatenate title and review straight away, use attention (letting the model decide), use weighted sum concatenation of final hidden states if title and review lengths differ significantly.

2.Avoid breaking sequential dependence by using same mask for each time step (variation can be across the depth and batches); Padding tokens should be 'masked' properly if to be ignored during loss computation.

3.Embedding layers can overfit because they're often large and trained from scratch;
Applying dropout to the embedding output adds noise, forcing the model to generalize rather than memorize exact embedding vectors;
Some researchers avoid dropout directly on embeddings when using pretrained embeddings (like GloVe/BERT), but it’s common in training embeddings from scratch.



In [36]:
print(len(Vocab)) #Parameter count shoots up! (Ideas behind using GloVe)
print(label_tensor)
print(title_tensor)
print(review_tensor)
print(df[df['Title'].apply(len)==0])
print(df.head(100))
print(df.index.size)

glove_embed={'UNK':1,'PAD':0}
embedding_dim=10

with open(glove_input_file,'r',encoding='utf8') as f:
  for line in f:
    parts=line.strip().split()
    word=parts[0]
    vector=np.array(parts[1:], dtype=np.float32)
    glove_embed[word] = vector

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from torch.optim.lr_scheduler import ReduceLROnPlateau
import matplotlib.pyplot as plt
import seaborn as sns

#Add batch normalisation, handle p
class RNN_model(nn.Module):
    def __init__(self, vocab, glove_map, embed_dim, hidden_dim, output_dim, pad_idx=0, dropout=0.25):

        super(RNN_model, self).__init__()
        embedding_matrix=self.create_embedding_matrix(vocab,glove_map,embed_dim)
        self.embedding=nn.Embedding.from_pretrained(torch.tensor(embedding_matrix,dtype=torch.float),freeze=True) #Can we have only selctive learning?

        layers=[]
        l_dim=embed_dim
        for dim in hidden_dim:
           layers.append(nn.RNN(l_dim, dim, bidirectional=True, batch_first=True))
           l_dim=2*dim
        self.rnn=nn.ModuleList(layers)
        self.fc=nn.Linear(4*hidden_dim[-1], output_dim)
        self.dropout=nn.Dropout(dropout)

    def create_embedding_matrix(self,vocab, glove_map, embed_dim):
      embedding_output=np.zeros((len(vocab),embed_dim))
      for word,index in vocab.items():
        if word in glove_map:
          embedding_output[index]=glove_map[word]
        else:
          if index==1:
            embedding_output[index]=np.zeros((embed_dim,))
          elif index==2:
            embedding_output[index]=np.random.uniform(0,0.5,embed_dim)
          else:
            embedding_output[index]=np.random.uniform(-0.5,0,embed_dim)
      return embedding_output


    def forward(self, title_batch, review_batch):
        title_output=self.embedding(title_batch)
        review_output=self.embedding(review_batch)
        if self.training:
          title_output=self.dropout(title_output)
          review_output=self.dropout(review_output)

        for layer in self.rnn:
          # fw_output_hidden,bw_output_hidden=layer(input)[1]

          # output,hidden=layer(input)
          # input=torch.cat((hidden[-2,:,:],hidden[-1,:,:]),dim=1)

          title_output,title_hidden=layer(title_output)
          review_output,review_hidden=layer(review_output)

        fw_final_title,bw_final_title=title_hidden[-2,:,:],title_hidden[-1,:,:]
        final_title=torch.cat((fw_final_title,bw_final_title),dim=1)

        fw_final_review,bw_final_review=review_hidden[-2,:,:],review_hidden[-1,:,:]
        final_review=torch.cat((fw_final_review,bw_final_review),dim=1)

        final_hidden=torch.cat((final_title,final_review),dim=1)

        if self.training:
          final_hidden=self.dropout(final_hidden)
        logits = self.fc(final_hidden)
        return logits
        # return torch.sigmoid(logits)+1

hidden_dim=[80,32,16]

RNN=RNN_model(Vocab,glove_embed,100,hidden_dim,1)
criterion=nn.BCEWithLogitsLoss()
optimiser=optim.Adam(RNN.parameters(),lr=0.001)
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
RNN=RNN.to(device)
criterion=criterion.to(device)

from tqdm import tqdm

def Back_PropTT(RNN, loaded_data, optimiser, criterion):
  RNN.train()

  max_epochs=16
  lr_min=1e-7
  loss_progression=[]
  scheduler=ReduceLROnPlateau(optimiser,mode='min',factor=0.1,patience=2, min_lr=lr_min, threshold=1e-3, threshold_mode='rel')

  epoch_count=1
  while epoch_count<=max_epochs:
    epoch_loss=0
    for label_batch, title_batch, review_batch in tqdm(loaded_data):

      predicted_output=RNN(title_batch.to(device),review_batch.to(device))
      loss=criterion(predicted_output.view(-1),label_batch.float().to(device))
      optimiser.zero_grad()
      loss.backward()
      optimiser.step()
      epoch_loss+=loss.item()*label_batch.shape[0]# Avoids graph build up or memory leak

    current_lr=optimiser.param_groups[0]['lr']
    print(f'current lr: {current_lr}')
    epoch_loss/=len(dataset)
    loss_progression.append(epoch_loss)
    scheduler.step(epoch_loss)

    print(f"Epoch:{epoch_count} Loss:{epoch_loss}")
    if(current_lr<=lr_min):
      print('Very slow learning rate')
      break
    if((epoch_count>2)and(int(loss_progression[-1]*1000)>int(loss_progression[-2]*1000))):
      for param_group in optimiser.param_groups:
        param_group['lr']*=0.1

    epoch_count+=1

  return loss_progression


101414
tensor([1, 1, 1,  ..., 1, 0, 1])
tensor([[    4,     5,     6,  ...,     0,     0,     0],
        [    7,     8,     9,  ...,     0,     0,     0],
        [   11,     0,     0,  ...,     0,     0,     0],
        ...,
        [   31,   213,   468,  ...,     0,     0,     0],
        [   58,   559,    89,  ...,     0,     0,     0],
        [17171,  3255,     0,  ...,     0,     0,     0]])
tensor([[ 1670,  2659,   480,  ...,     0,     0,     0],
        [   90,    32,  1852,  ...,     0,     0,     0],
        [    8,   805,   127,  ...,     0,     0,     0],
        ...,
        [  796, 27829,  1402,  ...,     0,     0,     0],
        [  155,  1246, 51405,  ...,     0,     0,     0],
        [21470,   666,    76,  ...,     0,     0,     0]])
Empty DataFrame
Columns: [Label, Title, Review]
Index: []
    Label                                              Title  \
0       1                  [tensor(4), tensor(5), tensor(6)]   
1       1      [tensor(7), tensor(8), tensor(9), t

In [ ]:
# What else to load on device

part_label_tensor,part_title_tensor,part_review_tensor=label_tensor[:20000],title_tensor[:20000],review_tensor[:20000]

dataset=TensorDataset(part_label_tensor, part_title_tensor, part_review_tensor)
loaded_data=DataLoader(dataset,batch_size=128,shuffle=True)

loss_vect=Back_PropTT(RNN, loaded_data, optimiser, criterion)
plt.plot(loss_vect)
plt.xlabel("Epoch")
plt.ylabel("Epoch_Loss")
plt.title("Loss variation against epoch")
plt.show()







100%|██████████| 157/157 [02:18<00:00,  1.14it/s]


current lr: 0.001
Epoch:1 Loss:0.5197260540008545


100%|██████████| 157/157 [02:55<00:00,  1.12s/it]


current lr: 0.001
Epoch:2 Loss:0.4672487483024597


100%|██████████| 157/157 [02:32<00:00,  1.03it/s]


current lr: 0.001
Epoch:3 Loss:0.4640035334587097


 50%|████▉     | 78/157 [01:11<01:05,  1.20it/s]

In [15]:
test_df=pd.read_csv('test.csv',header=None,nrows=10000)
test_df.rename(columns={0:'Label',1:'Title',2:'Review'},inplace=True)
test_df.dropna(inplace=True)
test_df=test_df.reset_index(drop=True)

test_df['Title']=test_df['Title'].parallel_apply(complete_preprocessor)
test_df['Review']=test_df['Review'].parallel_apply(complete_preprocessor)
test_df['Label']-=1

test_df=test_df[(test_df['Title'].apply(len)>0)&(test_df['Review'].apply(len)>0)]
test_df=test_df.reset_index(drop=True)

         index  Label                                              Title  \
0            0      2                                           Great CD   
1            1      2  One of the best game music soundtracks - for a...   
2            2      1                   Batteries died within a year ...   
3            3      2              works fine, but Maha Energy is better   
4            4      2                       Great for the non-audiophile   
...        ...    ...                                                ...   
299975  299995      2                                    Flip the Script   
299976  299996      1                              Below my expectations   
299977  299997      1             STINKS! I'm only RATING b/c I have to!   
299978  299998      2                                 nice for the price   
299979  299999      1                                    Not recommended   

                                                   Review  
0       My lovely Pat has o

In [16]:
int_tokens(test_df, Vocab)
test_label_tensor,test_title_tensor,test_review_tensor=seq_pad(test_df)

test_data=TensorDataset(test_label_tensor, test_title_tensor, test_review_tensor)
test_data=DataLoader(test_data,batch_size=128,shuffle=False)

from sklearn.metrics import confusion_matrix,average_precision_score

RNN.eval()
with torch.no_grad():
  prediction_on_test=[]
  final_loss=0
  for test_label_batch, test_title_batch, test_review_batch in test_data:
    predicted_output=RNN(test_title_batch.to(device),test_review_batch.to(device))
    prediction_on_test.extend(predicted_output)

    loss=criterion(predicted_output.view(-1),test_label_batch.float().to(device))
    final_loss+=loss.item()*test_label_batch.shape[0]

  final_loss/=len(test_data.dataset)
  print(f"Final Loss: {final_loss}")

  ConfusionMatrix=confusion_matrix(test_label_tensor,prediction_on_test)
  print(f"Confusion Matrix: {ConfusionMatrix}")

  sns.heatmap(ConfusionMatrix, annot=True, fmt='d', cmap='Blues', cbar=False)
  plt.xlabel('Predicted Output')
  plt.ylabel('Actual Output')
  plt.title('Confusion Matrix')
  plt.show()







NameError: name 'RNN' is not defined